In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path

sys.path.append(os.path.join(Path().resolve(), '../../..'))

In [2]:
import os
import glob
import numpy as np

In [3]:
%matplotlib inline
from IPython import display
import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation

import numpy as np
from tqdm import tqdm

In [9]:
!pip install scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 63.4 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.0 MB/s eta 0:00:00


In [10]:
import cv2
import copy
from skimage.color import rgb2gray
from skimage import img_as_float, img_as_ubyte

switch1 = 0
n_blur1 = 2
ksize1 = 5
th_size = 5
switch2 = 3
n_blur2 = 1
ksize2 = 5

blurs = ["blur", "GaussianBlur", "bilateralFilter", "medianBlur"]
def make_binary_image(img, switch1, n_blur1, ksize1, size, switch2, n_blur2, ksize2):
    _img_col = copy.deepcopy(img).astype(np.float32)
    
    method1 = blurs[switch1]
    for k in range(n_blur1):
        if method1 == "blur":
            _img_col = cv2.blur(_img_col, (ksize1,ksize1))
        elif method1 == "medianBlur":
            if ksize1 > 5:
                ksize1 = 5
            _img_col = cv2.medianBlur(src=_img_col, ksize=ksize1)
        elif method1 == "GaussianBlur":
            _img_col = cv2.GaussianBlur(_img_col, (ksize1,ksize1), sigmaX=3)
        elif method1 == "bilateralFilter":
            _img_col = cv2.bilateralFilter(src=_img_col, d=ksize1, sigmaColor=75, sigmaSpace=75)
        else:
            method1 = "None"

    _img_col = img_as_float(_img_col)
    _img_gray = rgb2gray(_img_col).astype(np.uint8)
        
    img_adap = cv2.adaptiveThreshold(_img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, size, 1)
    
    method2 = blurs[switch2]
    _img_bin = copy.deepcopy(img_adap).astype(np.float32)
    for k in (range(n_blur2)):
        if method2 == "blur":
            _img_bin = cv2.blur(_img_bin, (ksize2,ksize2))
        elif method2 == "medianBlur":
            if ksize2 > 5:
                ksize2 = 5
            _img_bin = cv2.medianBlur(src=_img_bin, ksize=ksize2)
        elif method2 == "GaussianBlur":
            _img_bin = cv2.GaussianBlur(_img_bin, (ksize2,ksize2), sigmaX=3)
        elif method2 == "bilateralFilter":
            _img_bin = cv2.bilateralFilter(src=_img_bin, d=ksize2, sigmaColor=75, sigmaSpace=75)
        else:
            method2 = "None"
    return np.max(_img_bin)-_img_bin


In [12]:
from dataset.COBOTTA.pose_processing import preprocess_pose

def preprocess_data(file_name_input, folder_name_save, flag_bin=False):
    
    # file_name = "{}/{}".format(folder_name, name.replace('.bag', '.npy'))
    print(file_name_input)
    data = np.load(file_name_input, allow_pickle=True).item()
    d_pose_quat = np.zeros_like(data["pose_quat"])
    d_pose_quat[1:] = data["pose_quat"][1:] - data["pose_quat"][:-1]
    norm = np.linalg.norm(d_pose_quat[:,:],axis=1)
    
    # idx_use = np.where(norm>1e-4)[0]
    idx_use = np.where(norm>-1)[0]
        
    output = dict()
    for key in data.keys():
        if ("high_resolution" in key):
            print(key)
            ims = data[key][idx_use]
            ims_128 = []
            ims_64 = []
            size = ims.shape[1:3]

            n_data = len(ims)
            for t in range(n_data):
                im_128 = cv2.resize(ims[t], (int(size[0]/2), int(size[1]/2)), interpolation=cv2.INTER_LINEAR)
                ims_128.append(im_128)

                im_64 = cv2.resize(ims[t], (int(size[0]/4), int(size[1]/4)), interpolation=cv2.INTER_LINEAR)
                ims_64.append(im_64)

            output[key.replace("_high_resolution", "_256")] = ims
            output[key.replace("_high_resolution", "_128")] = np.array(ims_128)
            output[key.replace("_high_resolution", "_64")] = np.array(ims_64)

        else:
            output[key] = data[key][idx_use]
    
    if flag_bin:
        output_bin = dict()
        for key in output.keys():
            if "image" in key:
                ims = output[key]
                ims_bin = []
                for t in range(len(ims)):
                    im_bin = make_binary_image(ims[t], switch1, n_blur1, ksize1, th_size, switch2, n_blur2, ksize2)
                    im_bin = (np.array(im_bin)/255).astype(np.uint8)
                    ims_bin.append(im_bin)
                output_bin[key+"_bin"] = np.expand_dims(np.array(ims_bin), axis=-1)
        for key in output_bin.keys():
            output[key] = output_bin[key]

    output["done"][-1] = 1
    if "weight_value" in output.keys():
        output["weight_value_norm"] = output["weight_value"]/2000
    
    output = preprocess_pose(output)
    
    basename = os.path.basename(file_name_input)
    save_file_name = "{}/{}".format(folder_name_save, basename)
    os.makedirs(os.path.dirname(save_file_name), exist_ok=True)
    np.save(save_file_name, output)


/home/docker/.pyenv/versions/3.8.0/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
folder_name = "cobotta_2022-05-29_point_drilling_npy_norm/train"
# folder_name = "dataset/20052022_drilling_npy/validation_episodes"

# file_names = glob.glob(os.path.join(folder_name, '*/*.npy'))
file_names = glob.glob(os.path.join(folder_name, '*.npy'))
n_episode = len(file_names)

print("find %d npy files!" % n_episode)


find 20 npy files!


In [14]:
folder_name

'cobotta_2022-05-29_point_drilling_npy_norm/train'

In [15]:
file_names = glob.glob(os.path.join(folder_name, '*.npy'))
# file_names

In [16]:
folder_name_save = "dataset/train"

for file_name in tqdm(file_names):
    preprocess_data(file_name, folder_name_save)


 15%|██████████████████████████▋                                                                                                                                                       | 3/20 [00:00<00:00, 28.17it/s]

cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-52-36.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-34-24.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-36-45.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-50-03.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-32-07.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-09-11-09.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-24-02.npy


 65%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                              | 13/20 [00:00<00:00, 40.06it/s]

cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-56-49.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-26-46.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-54-48.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-42-20.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-59-26.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-09-16-08.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-09-01-21.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-29-43.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-09-04-56.npy


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 38.68it/s]

cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-09-13-44.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-09-08-37.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-47-01.npy
cobotta_2022-05-29_point_drilling_npy_norm/train/cobotta_2022-05-29-08-39-28.npy


In [17]:
# folder_name_save

In [18]:
# file_names = glob.glob(os.path.join(folder_name_save, '*.npy'))

# Validation

In [19]:

folder_name = "cobotta_2022-05-29_point_drilling_npy_norm/validation"

# file_names = glob.glob(os.path.join(folder_name, '*/*.npy'))
file_names = glob.glob(os.path.join(folder_name, '*.npy'))
n_episode = len(file_names)

print("find %d npy files!" % n_episode)


find 3 npy files!


In [20]:
folder_name_save = "dataset/validation"

for file_name in tqdm(file_names):
    preprocess_data(file_name, folder_name_save)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 36.56it/s]

cobotta_2022-05-29_point_drilling_npy_norm/validation/cobotta_2022-05-29-09-20-03.npy
cobotta_2022-05-29_point_drilling_npy_norm/validation/cobotta_2022-05-29-09-21-55.npy
cobotta_2022-05-29_point_drilling_npy_norm/validation/cobotta_2022-05-29-09-18-10.npy


In [21]:
file_names = glob.glob(os.path.join(folder_name_save, '*.npy'))
# file_names = glob.glob(os.path.join(folder_name, '*.npy'))

In [22]:
dataset = []
for file_name in tqdm(file_names):
    # print(file_name)
    data = np.load(file_name, allow_pickle=True).item()
    dataset.append(data)
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 139.07it/s]
